# 生成所需数据

In [5]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [6]:
# 指定工作空间
env.workspace = r'F:\cache_data\shp_file\qz\ele_qz'
if not os.path.exists(env.workspace):
    os.makedirs(env.workspace)
env.overwriteOutput = True

In [7]:
# 三调地块路径
sd_data = r'F:\cache_data\shp_file\qz\qz_sd_polygon.shp'
# 母质路径
mz_data = r'F:\cache_data\shp_file\qz\dz\rock_type_result_20250219.shp'
# 坡位矢量路径
slope_data = r'F:\cache_data\shp_file\qz\qz_slopepostion_polygon.shp'

In [ ]:
# 提取耕林园草
select_name = 'qz_sd_select.shp'
arcpy.Select_analysis(sd_data,select_name,f"{'DLLB'} IN ('自然土','水稻土', '非自然土')")

In [ ]:
# 融合母质
dissolve_name = 'qz_mz_result_dissove.shp'
arcpy.Dissolve_management(in_features=mz_data,out_feature_class=dissolve_name,dissolve_field='MZMC',multi_part='SINGLE_PART')

In [ ]:
# 相交
intersect_name = 'qz_merge_data.shp'
arcpy.Intersect_analysis(in_features=[select_name,dissolve_name,slope_data],out_feature_class=intersect_name,join_attributes='ALL')    

In [ ]:
# 转单部件
single_name = 'qz_merge_data_single.shp'
arcpy.MultipartToSinglepart_management(intersect_name,single_name)